In [ ]:
!pip install tensorflow matplotlib seaborn scikit-learn gradio


In [ ]:
import zipfile


zip_path = "/content/garbage_dataset.zip"
extract_path = "/content/garbage_dataset"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Dataset extracted!")


✅ Dataset extracted!


In [ ]:
import os

print("Class folders inside dataset:")
print(os.listdir("/content/garbage_dataset"))


Class folders inside dataset:
['TrashType_Image_Dataset']


In [ ]:
import tensorflow as tf

image_size = (124, 124)
batch_size = 32
dataset_path = "/content/garbage_dataset/TrashType_Image_Dataset"

train_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=image_size,
    batch_size=batch_size
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=image_size,
    batch_size=batch_size
)

class_names = train_ds.class_names
print(" Classes:", class_names)


Found 2527 files belonging to 6 classes.
Using 2022 files for training.
Found 2527 files belonging to 6 classes.
Using 505 files for validation.
📂 Classes: ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']


In [ ]:
from tensorflow.keras.applications import EfficientNetV2B0
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Rescaling, GlobalAveragePooling2D, Dense

# Load EfficientNetV2B0 without top layer
base_model = EfficientNetV2B0(
    include_top=False,
    input_shape=(124, 124, 3),
    weights='imagenet'
)
base_model.trainable = False  # freeze base model

# Create full model
model = Sequential([
    Rescaling(1./255, input_shape=(124, 124, 3)),
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dense(len(class_names), activation='softmax')  # Output = no. of classes
])


24274472/24274472 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)


Epoch 1/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 50s 366ms/step - accuracy: 0.2082 - loss: 1.7722 - val_accuracy: 0.2040 - val_loss: 1.7302
Epoch 2/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - accuracy: 0.2186 - loss: 1.7380 - val_accuracy: 0.2040 - val_loss: 1.7468
Epoch 3/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.1962 - loss: 1.7456 - val_accuracy: 0.2455 - val_loss: 1.7196
Epoch 4/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.2358 - loss: 1.7247 - val_accuracy: 0.2455 - val_loss: 1.7211
Epoch 5/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.2333 - loss: 1.7160 - val_accuracy: 0.2455 - val_loss: 1.7127
Epoch 6/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.2151 - loss: 1.7345 - val_accuracy: 0.2396 - val_loss: 1.7277
Epoch 7/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.2322 - loss: 1.7224 - val_accuracy: 0.2515 - val_loss: 1.7258
Epoch 8/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.2119 - loss: 1.7350 - val_accuracy: 0.2455 -

In [ ]:
model.save("garbage_model.h5")
print("✅ Model saved successfully!")


✅ Model saved successfully!
